In [1]:
import os
os.environ['USE_PYGEOS'] = '0'
import matplotlib.pyplot as plt
plt.rcParams["font.family"] = "Arial"
plt.rcParams.update({'font.size': 20})

import pandas as pd
import geopandas as gpd

import numpy as np
import glob
import json
import seaborn as sns
import copy

import zipfile
from datetime import datetime

########## Check memory##########
import os
import psutil
process = psutil.Process(os.getpid())
print(process.memory_info().rss/1e6)

pd.set_option('display.max_colwidth', 25)

216.010752


# 1. calculate

In [ ]:
## area data is monthly lake area from the GLEV dataset (https://doi.org/10.1038/s41467-022-31125-6) 
## with updated area extraction algorithm from Zhao et al. (2020) (https://doi.org/10.1016/j.rse.2020.112104)
## ice data is monthly ice cover fraction from the GLEV dataset (https://doi.org/10.1038/s41467-022-31125-6)

In [2]:
area_data = np.load('out_0_area.npy', mmap_mode='r')
ice_data = np.load('out_0_ice.npy', mmap_mode='r')

lake_infos_path = 'HydroLakes/Hylak_info.npy'
lake_infos = np.load(lake_infos_path)

In [3]:
months1 = pd.date_range('1984-03-01','2020-12-31',freq='MS')
months2 = pd.date_range('1984-03-01','2020-12-31',freq='M')
months = [c1+(c2-c1)/2 for c1,c2 in zip(months1, months2)]
months_ref = [(c-np.datetime64('1984-01-01')).days for c in months]

print(months_ref[0:20])
print(len(months_ref))

days = pd.date_range('1984-03-01','2020-12-31',freq='D')
days_ref = [(c-np.datetime64('1984-01-01')).days for c in days]
print(days_ref[0:60])
print(len(days_ref))

[75, 105, 136, 166, 197, 228, 258, 289, 319, 350, 381, 410, 440, 470, 501, 531, 562, 593, 623, 654]
442
[60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119]
13455


In [4]:
day_df = pd.DataFrame([])
day_df['day'] = pd.date_range('1984-01-01','2020-12-31',freq='D')
day_df['month'] = day_df['day'].dt.strftime('%Y-%m-01')
day_df['mth'] = day_df['day'].dt.month
day_df['one'] = 1
mdays = day_df.groupby('month')['one'].apply(list).values
mdays = [np.array(c) for c in mdays]

In [5]:
day_df1 = day_df[day_df['day'] >= '1984-03-01'].copy().reset_index(drop=True)

idx = {}
for mth in np.arange(1,13,1):
    idx[mth] = day_df1[day_df1['mth'] == mth].index.values
print(idx)

{1: array([  306,   307,   308, ..., 13117, 13118, 13119]), 2: array([  337,   338,   339, ..., 13146, 13147, 13148]), 3: array([    0,     1,     2, ..., 13177, 13178, 13179]), 4: array([   31,    32,    33, ..., 13207, 13208, 13209]), 5: array([   61,    62,    63, ..., 13238, 13239, 13240]), 6: array([   92,    93,    94, ..., 13268, 13269, 13270]), 7: array([  122,   123,   124, ..., 13299, 13300, 13301]), 8: array([  153,   154,   155, ..., 13330, 13331, 13332]), 9: array([  184,   185,   186, ..., 13360, 13361, 13362]), 10: array([  214,   215,   216, ..., 13391, 13392, 13393]), 11: array([  245,   246,   247, ..., 13421, 13422, 13423]), 12: array([  275,   276,   277, ..., 13452, 13453, 13454])}


In [6]:
output = []
for k in range(1, len(area_data)):
    
    hylak_id = k + 1
        
    dates = days
    years = [c.year for c in dates]
    areas = area_data[k]
    
    ######### Daily ice cover ##########
    ice = ice_data[hylak_id-2]
    tmp = copy.deepcopy(mdays)
    for j in range(444):
        length = len(tmp[j])
        if ice[j] == 0:
            tmp[j] = np.zeros(length)
        elif ice[j] < 1 and j>=1 and j<443:
            if ice[j-1] == 1:
                tmp[j][int(length-(1-ice[j])*length):length] = np.zeros(length-int(length-(1-ice[j])*length))
            if ice[j+1] == 1:
                tmp[j][0:int((1-ice[j])*length)] = np.zeros(int((1-ice[j])*length))
        elif ice[j] < 1 and j==0:
            if ice[j+1] == 0:
                tmp[j][int(length-(1-ice[j])*length):length] = np.zeros(length-int(length-(1-ice[j])*length))
            if ice[j+1] == 1:
                tmp[j][0:int((1-ice[j])*length)] = np.zeros(int((1-ice[j])*length))
        elif ice[j] < 1 and j==443:
            if ice[j-1] == 0:
                tmp[j][0:int((1-ice[j])*length)] = np.zeros(int((1-ice[j])*length))
            if ice[j-1] == 1:
                tmp[j][int(length-(1-ice[j])*length):length] = np.zeros(length-int(length-(1-ice[j])*length))

    icefree = [1-num for sublist in tmp[2:] for num in sublist]
    
    ######### percentiles 10, 90 #########
    area_daily = np.interp(days_ref, months_ref, areas)

    p10 = {}
    p90 = {}
    for mth in np.arange(1,13,1):
        p10[mth], p90[mth] = np.quantile(area_daily[idx[mth]], [0.1,0.9]) 
        
    p10 = np.concatenate([mdays[i]*p10[i%12+1] for i in range(444)[2:]])
    p90 = np.concatenate([mdays[i]*p90[i%12+1] for i in range(444)[2:]])
    
    ######### area data extremes ##########
    lows = (area_daily <= p10).astype(int)
    highs = (area_daily >= p90).astype(int)
    
    lows = [lows[i]*icefree[i] for i in range(len(lows))]
    highs = [highs[i]*icefree[i] for i in range(len(highs))]
    
    low_intensity = [(p10[i]-area_daily[i])*lows[i] for i in range(len(area_daily))]
    high_intensity = [(area_daily[i]-p90[i])*highs[i] for i in range(len(area_daily))]
    
    year_idx = np.unique(years, return_index=True)[1][1:]
    year_yr = [np.mean(c) for c in np.split(years, year_idx)]
    icefree_yr = [np.sum(c) for c in np.split(icefree, year_idx)]
    low_yr = [np.sum(c) for c in np.split(lows, year_idx)]
    high_yr = [np.sum(c) for c in np.split(highs, year_idx)]
    
    low_inty_yr = [np.mean(c) for c in np.split(low_intensity, year_idx)]
    high_inty_yr = [np.mean(c) for c in np.split(high_intensity, year_idx)]
    hylak_id_yr = np.ones(len(year_yr))*hylak_id
    
    output.append([hylak_id_yr, year_yr, icefree_yr, low_yr, high_yr, low_inty_yr, high_inty_yr])
    
    if k%100 == 0:
        print(k, datetime.now(), process.memory_info().rss/1e6)
    
np.save('out_1_lake_area_extremes_ss.npy', np.float32(output))

# 2. summarize

In [7]:
output = np.load('out_1_lake_area_extremes_ss.npy', mmap_mode='r')
output.shape

In [8]:
df = pd.DataFrame(output.transpose(0,2,1).reshape(-1,7), 
                   columns=['hylak_id','year','icefree','lows','highs','low_inty','high_inty'])

def period(year):
    if year>=1985 and year<=1999:
        prd = 'X1980_90s'
    elif year>=2000 and year<=2009:
        prd = 'X2000s'
    elif year>=2010 and year<=2019:
        prd = 'X2010s'
    else:
        prd = 'Ot'
    return prd
df['period'] = df['year'].apply(period)

df_avg = df[(df['period'].isin(['X1980_90s','X2000s','X2010s']))] \
            .groupby(['hylak_id','period']).mean().drop(columns=['year']).reset_index()

In [9]:
df_low_freq = df_avg.pivot(index='hylak_id',columns='period',values='lows')
df_low_freq.columns = [c+'_LowEx_Freq' for c in df_low_freq.columns]

df_high_freq = df_avg.pivot(index='hylak_id',columns='period',values='highs')
df_high_freq.columns = [c+'_HighEx_Freq' for c in df_high_freq.columns]

df_low_int = df_avg.pivot(index='hylak_id',columns='period',values='low_inty')
df_low_int.columns = [c+'_LowEx_Int' for c in df_low_int.columns]

df_high_int = df_avg.pivot(index='hylak_id',columns='period',values='high_inty')
df_high_int.columns = [c+'_HighEx_Int' for c in df_high_int.columns]

In [10]:
all_data = pd.concat([df_low_freq, df_high_freq, df_low_int, df_high_int], axis=1).reset_index()
all_data.to_csv('out_1_lakeArea_extremes_ss.csv.zip', index=False, 
                compression=dict(method='zip', archive_name='out_1_lakeArea_extremes_ss.csv'))